In [ ]:
### Let us use Selenium to get all the data in the page
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import time


In [ ]:
# we need the url and the driver of our choice
# selenium has a lot of drivers, but we will use the firefox one
url = "https://www.getducked.com/collections/all"
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service)
driver.get(url)

In [ ]:
# Wait for the page to fully load
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(200)  # Adjust the sleep time as needed

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
soup_duck = BeautifulSoup(driver.page_source, 'html.parser')
product_cards = soup_duck.find_all('product-card')

In [ ]:
titles = []
urls = []
prices = []

for product_card in product_cards:
    # Get the product title
    title = product_card.find('a', class_='product-card-title').text.strip()
    titles.append(title)

    # Get the product URL
    href = product_card.find('a', class_='product-card-title')['href']
    urls.append(href)

    # Get the product price
    price = product_card.find(class_='money').text[2:].strip()
    prices.append(price)

In [ ]:
len(titles), len(urls), len(prices)

In [ ]:
prices = [float(price) for price in prices]
data = pd.DataFrame({'title': titles, 'url': urls, 'price(RM)': prices})
data.to_csv('ducked_sel.csv', index=False)